In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'svg'

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import os
import numpy as np
from torchsummary import summary
from torchvision import transforms                                                                                                                                        
from torchvision import models
import matplotlib as mpl
import torchvision
mpl.rcParams['figure.dpi'] = 600
from PIL import Image
import PIL
import matplotlib.pyplot as plt
import pandas as pd
import numpy

import sys
sys.path.append("../../extra/pytorch-cnn-visualizations/src")
import deep_dream

sys.path.append("../../src/")
sys.path.append("../")
from utils.visualizations.visualize import visualize
import model
from datasets import imagenet
from utils.function import recreate_image


from layer_activation_with_guided_backprop import GuidedBackprop
from misc_functions import save_gradient_images
from aux.utils import obtain_features_map, load_imgs, zscore, extract_valid
from aux.visualization import visualize_features_map
from aux.visualization import visualize_features_map_for_comparision
from utils.visualizations.visualize import concat_imgs, preprocess_arrays

# Model

In [3]:
# model
backbone = "vgg16"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = model.Network(backbone=backbone, num_classes=1000)
net.to(device)
# resume from model
resume = "037-0"
model_dir = "/home/lincolnzjx/Desktop/Interpretation/saved/models"
resume_exp = resume.split("-")[0]
resume_epoch = resume.split("-")[1]
print("Resume from model from exp: {} at epoch {}".format(resume_exp, resume_epoch))
resume_path = os.path.join(model_dir, str(resume_exp), str(resume_epoch))
ckpt = torch.load(resume_path, map_location=device)
net.load_state_dict(ckpt, strict=False) 

Entire model.
Resume from model from exp: 037 at epoch 0


<All keys matched successfully>

# GBP

In [4]:
def gbp(prep_img, pretrained_model, selected_layer, selected_filter):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    GBP = GuidedBackprop(pretrained_model) 
    prep_img.requires_grad_(True)
    guided_grads = GBP.generate_gradients(prep_img, None, selected_layer, selected_filter)
    return guided_grads

In [5]:
def main(selected_layer=None, selected_filter=None, class_index=None, 
         img_index=None):
    num_class = 100
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    reverse_mean = [-0.485, -0.456, -0.406]
    reverse_std = [1/0.229, 1/0.224, 1/0.225]
    train_transform = transforms.Compose([                                                                                                                                
        transforms.Resize((224, 224), interpolation=Image.BILINEAR),                                                                                              
        transforms.ToTensor(),                                                                                                                                            
           ])         
    trainset = imagenet.ImageNet(root="/media/lincolnzjx/HardDisk/Datasets/", 
                                 is_train=True, transform=train_transform)
    trainset.set_data([class_index], num_class)
    imgs_path = []                                                                                                                                                            
    images = []
    labels = []
    
    for index, (img, label, img_path) in enumerate(trainset):                                                                                                                                     
        if index == img_index:
            images.append(img.unsqueeze(0))                                                                                                                                       
            labels.append(label)                                                                                                                                                  
            imgs_path.append(img_path)  
    
    dirs = "/home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/"
    dirs = os.path.join(dirs, str(class_index))
    
    dir_name = "opt"
    dir_name_fm = "fm"
    
    dir_path = os.path.join(dirs, dir_name)
    dir_path_fm = os.path.join(dirs, dir_name_fm)
    
    os.makedirs(dir_path, exist_ok=True)
    os.makedirs(dir_path_fm, exist_ok=True)
    
    mean = torch.Tensor([0.485, 0.456, 0.406]).reshape(1,-1,1,1)
    std  = torch.Tensor([0.229, 0.224, 0.225]).reshape(1,-1,1,1)
    
    for img, img_path in zip(images, imgs_path):
        name = img_path.split("/")[-1].split(".")[0]
        img = img[0].permute((1,2,0)).numpy()
        X = (torch.FloatTensor(img[numpy.newaxis].transpose([0,3,1,2])*1) - mean) / std
        
        # Ori
        ori_activation_maps = net.get_activation_maps(X.cuda(), selected_layer)
        
        sel_feature_map = ori_activation_maps[0][0,selected_filter]
        min_val = sel_feature_map.min()
        max_val = sel_feature_map.max()
        sel_feature_map -= sel_feature_map.min()
        sel_feature_map /= sel_feature_map.max()
        ori_fm_path = os.path.join(dir_path_fm, "ori_" + name +".png")
        cm = plt.get_cmap("jet")
        sel_feature_map = cm(sel_feature_map)
        sel_feature_map = (sel_feature_map[:, :, :3] * 255).astype(np.uint8)
        
        # GBP 
        pretrained_model = torchvision.models.vgg16(pretrained=True)
        guided_grads = gbp(X.clone().detach(), pretrained_model, selected_layer, selected_filter)
        print(guided_grads.shape)
        name = ("layer_{}_filter_{}_".format(selected_layer, selected_filter)) + name
        path = os.path.join(dir_path, name+".png")
        
        dir_path_fm_sub = os.path.join(dir_path_fm, name)
        os.makedirs(dir_path_fm_sub, exist_ok=True)
        fm_name = "feature_map_filter_{}".format(selected_filter)
        fm_path = os.path.join(dir_path_fm_sub, fm_name +".png")
        
        # Standalize
        # relevance_img = standalize(R[0][0], X.numpy(), path)
        # fm = visualize_sel_fm(net, relevance_img.transpose(2,0,1), selected_layer, selected_filter, min_val, max_val)
        # fm = (255 * fm).astype(np.uint8)
        # Image.fromarray(fm).save(fm_path)
        
        # Standlize remove negative
        # relevance_img = standalize_remove_negative(R[0][0], X.numpy(), path)
        # fm = visualize_sel_fm(net, relevance_img.transpose(2,0,1), selected_layer, selected_filter, min_val, max_val)
        # fm = (255 * fm).astype(np.uint8)
        # Image.fromarray(fm).save(fm_path)
        
        # Scale positive
        # relevance_img = scale_positive(R[0][0], X.numpy(), path)
        # fm = visualize_sel_fm(net, relevance_img.transpose(2,0,1), selected_layer, selected_filter, min_val, max_val)
        # fm = (255 * fm).astype(np.uint8)
        # Image.fromarray(fm).save(fm_path)
        
        # Scale total
        relevance_img = scale_total(guided_grads, X.cpu().numpy()[0], path)
        fm = visualize_sel_fm(net, relevance_img, selected_layer, selected_filter, min_val, max_val)
        fm = (255 * fm).astype(np.uint8)
        
        # plt.imshow(fm)
        plt.savefig(fm_path)
        
        img = np.array(Image.open(path))
        X = (torch.FloatTensor(img[numpy.newaxis].transpose([0,3,1,2])*1) - mean) / std
        print(X.shape)
        opt_activation_maps = net.get_activation_maps(X.contiguous().cuda(), selected_layer)

        visualize(ori_activation_maps, opt_activation_maps,
                  img_index=img_index, layer_name=selected_layer,
                  backbone="vgg16", num_class=num_class,
                  exp="GBP/{}".format(class_index), imgs_path=[dir_path_fm_sub])

# Visualization

In [6]:
def visualize_sel_fm(net, relevance_img, selected_layer, selected_filter, min_val, max_val): 
    cm = plt.get_cmap("jet")
    opt_activation_maps = net.get_activation_maps(torch.FloatTensor(relevance_img).unsqueeze(dim=0).cuda(), selected_layer)
    opt_sel_feature_map = opt_activation_maps[0][0,selected_filter]
    opt_sel_feature_map -= min_val
    opt_sel_feature_map = np.maximum(0, opt_sel_feature_map)
    opt_sel_feature_map /= (opt_sel_feature_map.max() + 1e-8)
    opt_sel_feature_map = np.minimum(1, opt_sel_feature_map)
    opt_sel_feature_map = cm(opt_sel_feature_map)
    return opt_sel_feature_map

## Scale max

## Standarization

In [7]:
# relevance = np.array(guided_grads)
# normalized_img = images_zscore[0]
# relevance_img = relevance * normalized_img
# new_image = recreate_image(relevance_img, reverse_mean, reverse_std)
# print(relevance.max(), relevance.min())
# print(new_image.max(), new_image.min())
# Image.fromarray(new_image)

In [8]:
# def scale_max(relevance, normalized, path=None):
#     reverse_mean = [-0.485, -0.456, -0.406]
#     reverse_std = [1/0.229, 1/0.224, 1/0.225]
#     
#     relevance = np.array(relevance)
#     relevance /= (np.abs(relevance)).max()
#     relevance_img = normalized * relevance
#     new_image = recreate_image(relevance_img, reverse_mean, reverse_std)
#     print(new_image.max(), new_image.min())
#     Image.fromarray(new_image).save(path)
#     return relevance_img

## Scale total with zero

In [9]:
# def scale_total_with_zero(relevance, normalized, path=None):
#     reverse_mean = [-0.485, -0.456, -0.406]
#     reverse_std = [1/0.229, 1/0.224, 1/0.225]
#     
#     relevance = np.array(relevance)
# #     relevance_img = normalized * (relevance * relevance==0)
#     relevance -= relevance.min()
#     relevance /= relevance.max()
#     relevance_img = relevance_img * relevance
#     
#     new_image = recreate_image(relevance_img, reverse_mean, reverse_std)
# #     print(new_image.max(), new_image.min())
#     Image.fromarray(new_image).save(path)
#     return relevance_img

## Scale total

In [10]:
def scale_total(relevance, normalized, path=None):
    reverse_mean = [-0.485, -0.456, -0.406]
    reverse_std = [1/0.229, 1/0.224, 1/0.225]
    
    relevance = np.array(relevance)
    relevance /= np.max(np.abs(relevance))
    relevance_img = normalized * relevance
    new_image = recreate_image(relevance_img, reverse_mean, reverse_std)
    Image.fromarray(new_image).save(path)
    return relevance_img

# Main

In [11]:
csv_file = pd.read_excel("../filter.xlsx", header=None)
csv_file = csv_file.values

In [ ]:
dicts = {
     948: [1, 3],
     444: [13, 20],
     522: [14,21],
     14: [25, 3],
     84: [29,11]
}
class_index = None
for item1, item2, _ in csv_file:
    if isinstance(item1, str) and item1.startswith("class"):
        class_index = int(item1.replace("class", ""))
        continue
    # if class_index != 948:
    #     continue
    selected_filter = int(item1)
    selected_layer = int(item2)
    if class_index == 948 and (selected_layer == 1 or selected_layer == 3):
        continue
    # img_index1, img_index2 = item3.replace("[", "").replace("]", "").split(",")
    # img_index1 = int(img_index1)
    # img_index2 = int(img_index2)
#     img_index1 = dicts[class_index][0]
#     img_index2 = dicts[class_index][1]
#     print(class_index, selected_layer, selected_filter, img_index1, img_index2)
#     main(selected_layer, selected_filter, class_index, img_index=img_index1)
    for i in range(100):
        main(selected_layer, selected_filter, class_index, img_index=i)

Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has been created /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/
Plot mode is => img_scale
Color map is => jet


../aux/visualization.py:123: UserWarning:

This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.

findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.


Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_53_n07742313_10026-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has been created /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/
Plot mode is => img_scale
Color map is => jet
Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_53_n07742313_10027-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has been created /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/
Plot mode is => img_scale
Color map is => jet
Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_53_n07742313_10036-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has

../aux/visualization.py:55: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_53_n07742313_10419-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has been created /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/
Plot mode is => img_scale
Color map is => jet
Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_53_n07742313_10424-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has been created /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/
Plot mode is => img_scale
Color map is => jet
Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_53_n07742313_1043-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has 

...

Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_53_n07742313_10990-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has been created /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/
Plot mode is => img_scale
Color map is => jet
Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_53_n07742313_10992-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has been created /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/
Plot mode is => img_scale
Color map is => jet
Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_53_n07742313_10994-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has

Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_64_n07742313_10170-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has been created /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/
Plot mode is => img_scale
Color map is => jet
Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_64_n07742313_10173-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has been created /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/
Plot mode is => img_scale
Color map is => jet
Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_64_n07742313_1018-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has 

Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_64_n07742313_10480-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has been created /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/
Plot mode is => img_scale
Color map is => jet
Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_64_n07742313_10488-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has been created /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/
Plot mode is => img_scale
Color map is => jet
Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_64_n07742313_10490-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has

Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_64_n07742313_10690-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has been created /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/
Plot mode is => img_scale
Color map is => jet
Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_64_n07742313_10692-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has been created /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/
Plot mode is => img_scale
Color map is => jet
Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_64_n07742313_10712-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has

Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_64_n07742313_10882-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has been created /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/
Plot mode is => img_scale
Color map is => jet
Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_64_n07742313_10884-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has been created /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/
Plot mode is => img_scale
Color map is => jet
Successfully Save pdf to /home/lincolnzjx/Desktop/Interpretation/saved/generated/GBP/948/feature_map/layer-6-layer_6_filter_64_n07742313_10892-Comparision.png
Len of new dataset is :100
(3, 224, 224)
torch.Size([1, 3, 224, 224])
Directory has

In [ ]:
selected = [
    [1, 47],
    [1, 16],
    [3, 20],
    [3, 41],
    [6, 114],
    [6, 76],
    [8, 17],
    [8, 99],
    [11, 174],
    [11, 75],
    [13, 98],
    [13, 21],
    [15, 102],
    [15, 173],
    [18, 458],
    [18, 353],
    [20, 46],
    [20, 103],
    [22, 214],
    [22, 164],
    [25, 19],
    [25, 1],
    [27, 161],
    [27, 476],
    [29, 64],
    [29, 101]
]
class_indexes = [950]

for class_index in class_indexes:
    for selected_layer, selected_filter in selected:
        for i in range(100):
            main(selected_layer, selected_filter, class_index, img_index=i)